Welcome! The goal of this notebook is to familiarize economists with the numerical optimization techniques often required for quantitative work, as well as a fairly blatant attempt to sway others towards doing such work in open source software such as Python/Pyomo (or Julia/JuMP, which is faster but I think less flexible in the types of nonlinear problems it can easily adapt to) and combine resources.

In this notebook, I'll introduce some problems you may encounter in classwork and how you would tackle such problems using numerical optimization (even if the problems may have direct solutions), explaining the concepts as they arise.

### Prerequisites 
- Python installation of 2.7 or 3.4+, I recommend Anaconda https://www.anaconda.com/products/individual
- Pyomo, which can be installed from the directions at https://pyomo.readthedocs.io/en/stable/installation.html
- Optimization solvers such as ipopt -- ```conda install -c conda-forge ipopt``` (Note: you may have to debug this, I had to debug a bit on Ubuntu but directions are platform specific). You can install this without Anaconda, but it is extremely complicated. Using conda is **strongly** recommended. 
- Numpy, ```conda install numpy```


### Problem 1. Solving for an optimal consumption bundle / simple nonlinear programming problem

Consider the consumer's maximization problem of 
$$\max_{\{x_i\}_{i=1}^N} U(x_1,\dots,x_n)$$ subject to the standard budget constraint that $$\sum_i p_ix_i \leq w.$$

Suppose the number of goods is N=2.

Let's begin by assuming $U(\cdot)$ is Cobb Douglas with equal shares, i.e. $U(x_1,x_2)=x_1^{1/2}x_2^{1/2}$. Let's also assume that $p_1=2$, $p_2=1$ and $w=10$. Solving for the optimal consumption bundle using standard techniques, $x_1^*=(1/2)\times\frac{w}{p_1}=2.5$ and $x_2^*=5$. But let's pretend for now that this was particularly difficult and we needed to turn to the computer to solve this.


In [1]:
import numpy as np
import pyomo.environ as pyo ### imports pyomo environment
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition ### imports solvers
opt = SolverFactory('ipopt') ### Free solver good for nonlinear problems

model = pyo.ConcreteModel() ### define an initial model. 
# Note that this model is concrete because we initialize it with real, numeric values for prices
# See: https://pyomo.readthedocs.io/en/stable/pyomo_overview/abstract_concrete.html

N = 2 # number of goods
model.N = N # set this parameter for the model 
model.i = pyo.RangeSet(1, model.N) # create a set for these goods (goods in range 1 to N)

model.alpha = pyo.Param(model.i,within=pyo.NonNegativeReals,default=1.0/N) # set alpha as a vector of parameters 
# alpha is indexed by the set i
# within = fixes the parameter domain to the nonnegative reals
# default: set a default value to 1/N

model.w = 10
model.p = {1:2,2:1} # dictionary of prices
model.x = pyo.Var(model.i,within=pyo.NonNegativeReals,initialize=1.0)
# set variables, indexed by the set i. These are ultimately set by the optimization process
# set domain with within
# give a start value to start the optimization

def CobbDouglasUtility(model):
    # define the utility function, multiplying each x^alpha
    # import the model so we can grab the different objects we defined above from it
    return pyo.prod(model.x[i]**model.alpha[i] for i in model.i)

def BudgetConstraint(model):
    # define the budget constraint, adding up each p*x and setting <= w
    return(sum(model.p[i]*model.x[i] for i in model.i) <= model.w)

# set objective function
model.objective = pyo.Objective(rule=CobbDouglasUtility, sense=-1) ### last argument sets objective to maximize; minimize is the default

# set constraint
model.BC = pyo.Constraint(rule=BudgetConstraint)

# call ipopt to solve
results = opt.solve(model)

In [2]:
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
     model.display()
elif results.solver.termination_condition == TerminationCondition.infeasible:
     print ("This is an infeasible problem")
else:     # something else is wrong
    print(str(results.solver))

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 : 2.5000000232267734 :  None : False : False : NonNegativeReals
          2 :     0 : 5.0000000464532866 :  None : False : False : NonNegativeReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 3.535533938780264

  Constraints:
    BC : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 10.000000092906834 :  10.0


Notice here that the optimal values of $x_1$ and $x_2$ are somewhat above the optimal values we calculated. This is because the solver aims to maximize the objective while holding the constraint within some small tolerance of precision. You can adjust this using the parameter ```comparison_tolerance_for_fixed_vars``` (which is set to 1e-05 as a default). In general, adjusting the tolerance would come at the expense of computation time, and I don't foresee the need in most applications to mess around here.

In [104]:
# note that above you can see that the child "x" of "model" has 2 keys,
# 1 and 2. This gives us the hint that x is a dictionary that we can 
# subset just like any other!

x1 = np.round(model.x[1].value,4) ### enough to round off any tolerance errors
x2 = np.round(model.x[2].value, 4)
print("Optimal x1 is",x1,"and optimal x2 is",x2)

Optimal x1 is 2.5 and optimal x2 is 5.0


In [11]:
# getting the max value of the objective function:
model.objective()

3.535533938780264

In [12]:
# getting fulfilled value of the budget constraint:
model.BC()

10.000000092906834

Now, the beauty of Pyomo is that we can easily adjust the parameters of the model and resolve. 

Suppose now that $N=3$, and $U(\cdot)$ is quasilinear, i.e. $U(x_1,x_2,x_3)=x_1+\sqrt{x_2\times x_3}$. And now suppose that $\hat{p} = (2,1,1)$.

In [13]:
model = pyo.ConcreteModel() ### define an initial model
N = 3
model.N = N
model.i = pyo.RangeSet(1, model.N)
model.p = {1:2,2:1,3:1}
model.x = pyo.Var(model.i,within=pyo.PositiveReals,initialize=1.0)
model.w = 10


def QLinearUtility(model):
    return model.x[1]+(model.x[2]*model.x[3])**(1/2)

model.objective = pyo.Objective(rule=QLinearUtility, sense=-1)
model.BC = pyo.Constraint(rule=BudgetConstraint)
results = opt.solve(model)

In [14]:
x1 = np.round(model.x[1].value, 4) ### enough to round off any tolerance errors
x2 = np.round(model.x[2].value, 4)
x3 = np.round(model.x[3].value, 4)

print("Optimal x1 is",x1,"optimal x2 is",x2,"optimal x3 is",x3)

Optimal x1 is 3.1099 optimal x2 is 1.8901 optimal x3 is 1.8901


Notice that in order to change a few parameters and functional forms, we had to create a whole new model. However, Pyomo provides another class which makes changing data quite a bit easier.

In [191]:
N =  3
model = pyo.AbstractModel()

model.N = pyo.Param()
model.i = pyo.RangeSet(1, model.N)
model.p = pyo.Param(model.i,within=pyo.PositiveReals)
model.x = pyo.Var(model.i,within=pyo.PositiveReals,initialize=1.0)
model.w = pyo.Param(default=10)

model.objective = pyo.Objective(rule=QLinearUtility, sense=-1)
model.BC = pyo.Constraint(rule=BudgetConstraint)

Note that the above code is fairly similar to before, except for that we haven't defined any data at the time of creation. To do so, we create an instance which will read from a Python dict of the following form:

In [192]:
data = {None: {
    'N':{None: 3},
    'p':{1:2,2:1,3:1}
}}
instance = model.create_instance(data)
results = opt.solve(instance) ### for more information on solver output, use tee=True

One difference worth noting is that the variable values are now stored in ```instance```. Above, I have overwritten the default value for N to 3, but left alpha and w undefined, as the default value of 10 is fine. By using the initialize and default args above, we can use the AbstractModel() class very similarly to a ConcreteModel(), with the difference being that we need to create an instance first before solving with an AbstractModel(). 

In [193]:
x1 = np.round(instance.x[1].value, 4) ### enough to round off any tolerance errors
x2 = np.round(instance.x[2].value, 4)
x3 = np.round(instance.x[3].value, 4)

print("Optimal x1 is",x1,"optimal x2 is",x2,"optimal x3 is",x3)

Optimal x1 is 3.1099 optimal x2 is 1.8901 optimal x3 is 1.8901


Suppose we again wanted to change the utility function back to Cobb Douglas and change prices to $\hat{p}=(1,1,1)$, then we would simply run:

In [194]:
model.del_component(model.w) ### There's no need to delete w here, I'm just showing how you can
model.objective.deactivate() ### deactivate old objective function which is q-linear

model.w = pyo.Param(default=10)
model.alpha = pyo.Param(model.i,within=pyo.NonNegativeReals,default=1.0/N)
model.new_objective = pyo.Objective(rule=CobbDouglasUtility, sense=-1)

data[None]['p']={1: 1, 2: 1, 3: 1} ### change prices in dict
instance = model.create_instance(data)
results = opt.solve(instance) 

x1 = np.round(instance.x[1].value, 4) ### enough to round off any tolerance errors
x2 = np.round(instance.x[2].value, 4)
x3 = np.round(instance.x[3].value, 4)

print("Optimal x1 is",x1,"optimal x2 is",x2,"optimal x3 is",x3)

Optimal x1 is 3.3333 optimal x2 is 3.3333 optimal x3 is 3.3333


### Problem 2. Solving for parameter values of a workhorse trade model (the Armington model)

Consider the following setup: There are $1,\dots,N$ countries in the world. Consumers in each country consume a varieties of a traded good using CES demand an with elasticity of substitution over goods denoted $\sigma$. Labor is the only factor of production which is inelasticity supplied and assumed to be immobile. Trade between countries is costly (iceberg trade costs between i and j are denoted $\tau_{ji} \geq 1$ ) and the no-arbitrade condition applies, meaning that if $p_j$ is the price of a good in country $j$ and $p_{ji}$ is the price of country $j$'s output in country $i$, then we must have that $p_{ji}=\tau_{ji} p_j$. Each country produces a different variety of the good, known as the Armington assumption. Locations are characterized by productivities/marginal productivities of labor $A_i$. 

The main equation that defines the equilibrium of the Armington model is the goods market clearing condtion, or more simply that import demand = export supply:
\begin{equation}
w_iL_i = \sum_{j=1}^N w_jL_j \pi_{ji},
\end{equation}
where $\pi_{ji}$ is the share of expenditure country $i$ spends on goods from country $j$. In short, this basically says that consumer spending on goods from different countries (the RHS) cannot exceed their income (the LHS), for all countries $i$.

CES demand tells us that the share of expenditure country $i$ spends on goods from country $j$ is given by
$\pi_{ji} =X_{ji}/Y_i= p_{ji}^{1-\sigma} P_i^{\sigma-1}$, where $P_i^{1-\sigma}=\sum_l p_{li}^{1-\sigma}$. But if production of each good in each country is perfectly competitive, then $p_i=w_i/A_i$. Therefore, the price in country $i$ of consuming one unit from country $j$ is:
$p_{ji}=\tau_{ji}\frac{w_j}{A_j}$. Therefore, $$\pi_{ji}= \frac{\left(\tau_{ji}\frac{w_j}{A_j}\right)^{1-\sigma}}{\sum_l \left(\tau_{li}\frac{w_l}{A_l}\right)^{1-\sigma}}.$$

#### (Jump to here if you only care about the objective statement) This implies that the main equilibrium clearing condition is given by 
\begin{equation}
w_iL_i = \sum_{j=1}^N w_jL_j\frac{\left(\tau_{ji}\frac{w_j}{A_j}\right)^{1-\sigma}}{\sum_l \left(\tau_{li}\frac{w_l}{A_l}\right)^{1-\sigma}}, \forall i=1,\dots,N.
\end{equation}

Reformulating this for simplicity (which greatly helps in computation), we obtain

$$w_iL_iP_i^{1-\sigma} = \sum_{j=1}^N w_jL_j \left(\tau_{ji}\frac{w_j}{A_j} \right)^{1-\sigma}, \forall i=1,\dots,N,$$

with $P_i^{1-\sigma}=\sum_l \left(\tau_{li}\frac{w_l}{A_l}\right)^{1-\sigma}$.
Suppose we have information on the wages of workers in each country ($w_i$) and its total population stock ($L_i$), as well as the trade costs between each country pair ($\tau_{ji}$) and the elasticity of substitution ($\sigma=6$), but we don't know the productivities of each country $A_i$. The equilibrium clearing condition above provides us $N$ equations (i.e. one market clearing condition for each country $i$), which should be sufficient to solve for the N unknowns $A_1, \dots, A_N$. But, you'll notice that $A_1, \dots, A_N$ do not admit a closed form solution, and so we need to solve for these values using numerical optimization. These values will $\textbf{not}$ be unique, but they will be unique up to scale, so we can set the sum of the productivities equal to a normalizing constant (the choice of this will be important).

I've skipped alot of details because they're not really important here. But for more information on the setup of this model, see here:
https://scholar.princeton.edu/sites/default/files/zidar/files/connecting_theory_empirics_allen_gravity.pdf

In [2]:
sum_productivities = 1000.0 ### You might need to tweak this up/down to achieve convergence
productivity_initial_guess = 5.0 ### Same here
model = pyo.AbstractModel()

model.i = pyo.Set()
model.w = pyo.Param(model.i)
model.L = pyo.Param(model.i)
model.tau = pyo.Param(model.i,model.i)
model.A = pyo.Var(model.i,domain=pyo.NonNegativeReals, initialize=productivity_initial_guess)
model.sigma = 6.0

def p_ji(model,j,i):
    return (model.tau[j,i]*model.w[j]/model.A[j])**(1-model.sigma)
    
def Pi(model,i):    
    return sum((model.tau[l,i]*model.w[l]/model.A[l])**(1-model.sigma) for l in model.i)

def goods_market_clearing(model):
    return sum((model.w[i]*model.L[i]*Pi(model,i)-sum(model.w[j]*model.L[j]*p_ji(model,j,i) for j in model.i))**2.0 for i in model.i)

def goods_market_clearing_one_good_hold(model,i):
    return(model.w[i]*model.L[i]*Pi(model,i)==sum(model.w[j]*model.L[j]*p_ji(model,j,i) for j in model.i))

def productivity_sum(model):
    return(sum([model.A[i] for i in model.i])==sum_productivities)

model.objective = pyo.Objective(rule=goods_market_clearing, sense=1) ### sense = 1 means minimize
model.a_fix = pyo.Constraint(rule=productivity_sum)
model.one_good_fix = pyo.Constraint(model.i,rule=goods_market_clearing_one_good_hold) ### imposes constraint for all goods

What if your data (i.e. parameter values) is stored in a csv file? Read https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/dataportals.html for more info, but an example is provided below using the two csv files "wages_pop.csv" and "tau.csv"

In [3]:
data = pyo.DataPortal()
data.load(filename='wages_pop.csv', param=(model.w,model.L), index=model.i)
data.load(filename='tau.csv', param=model.tau, format='array')
instance = model.create_instance(data)

To begin, let's try deactivating all of the constraints that trade must be balanced in each market and try solving the model.

In [8]:
for i in range(1,201): 
      instance.one_good_fix[i].deactivate()
results = opt.solve(instance,tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      200
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    20100

Total number of variables............................:      200
                     variables with only lower bounds:      200
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

This results in finding an "optimal solution", but notice that the resulting objective function here is still quite large. 

In [7]:
instance.objective()

1986.9253349838589

In general, we can do a lot better when we add back in those trade balance constraints. While messing around with constraints, you may also be tempted to relax the constraint ```model.a_fix```, which will yield a lower objective function -- but notice that doing this yields variable solutions that we do not want, as the solver will simply choose very small values of $A$ in order to minimize the objective function. This shows the importance of double checking your solution values to see if they make intuitive sense.

In [9]:
### Lets re-activate those constraints
for i in range(1,201): 
      instance.one_good_fix[i].activate()

To solve problems like this, I find that it is necessary to run the solver mutliple times, until the solver reaches a point at which the objective function is sufficiently small (how small is sufficiently small depends on the problem of interest). Often times you will need to restart the algorithm using the solution values reached by a prior run, which can be done simply by running ```opt.solve(instance)``` without redefining the instance. As such, be prepared that you will need to run the code block below potentially several times. I find that it eventually converges to a point of local infeasibility (with an objective of 9.4329010e+02), but at this level, the solution is good enough.

In [10]:
opt.options['max_iter'] = 200 ### You can set this as you like, I find sometimes restarting the solver 
### yields quicker results, but too few iterations can cause the solver to hang up on a bad local solution

### Our objective function is defined as the squared sum of all lhs and rhs errors
### And our constraints are simply that the lhs and rhs equal for each i
### However, by Walras' law, one market is solved automatically, and objective acts as solving one market,
### so we need to omit two market constraints
for i in range(1,3): 
      instance.one_good_fix[i].deactivate()
results = opt.solve(instance,tee=True)

Ipopt 3.13.2: max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    39800
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    20100

Total number of variables............................:      200
                     variables with only lower bounds:      200
                variables with lower and upper bounds:        0
                     variables with only upper bounds:  

Now let's display retrieved solutions by looping over all variable values:

In [11]:
for i in range(1,200+1):
    print(np.round(instance.A[i].value,3))
### If you converge to where I do, first+second values should be 4.762 + 3.157

4.762
3.157
5.259
4.693
4.762
3.394
4.292
6.103
3.248
3.995
4.603
3.763
4.646
3.131
5.45
4.909
4.515
4.789
4.345
4.989
5.839
4.779
6.853
5.957
4.725
3.837
3.584
5.108
4.061
4.658
4.714
4.338
3.935
3.3
4.739
4.457
5.008
4.437
4.957
4.007
4.797
3.675
4.256
4.659
4.18
4.395
4.786
2.885
6.555
6.346
4.05
5.257
3.019
3.515
4.101
4.69
4.263
3.194
3.509
4.884
4.773
3.227
4.304
3.977
3.62
5.629
4.093
4.709
5.066
6.499
3.484
4.652
5.546
4.733
4.348
5.524
4.137
3.383
3.186
4.269
3.919
4.817
4.791
3.169
4.316
4.297
3.335
4.268
4.509
4.569
4.86
4.841
3.311
4.481
3.561
4.158
2.398
5.915
1.812
3.661
4.603
2.335
3.817
4.967
5.556
3.653
3.295
4.012
4.827
4.53
3.196
5.159
6.401
4.649
4.541
4.543
2.567
4.653
4.56
4.702
5.462
5.213
4.227
6.328
3.922
3.761
5.913
4.628
3.603
4.675
5.148
4.967
3.828
5.373
2.864
5.679
6.393
5.009
5.985
5.067
3.146
4.917
4.613
4.702
3.785
4.77
4.672
3.896
4.499
4.65
3.669
5.502
6.297
8.912
4.8
6.309
4.533
5.457
5.153
3.816
6.329
4.862
4.764
3.854
4.22
4.665
3.494
4.073
3.617
4

Now, let's check what these implied productivities yield for trade balances for each country and if we think that they are good enough.

In [12]:
### Redefine these functions to take stored parameter values as arguments, rather than the model class
def p_ji_check(j,i):
    return (tau[j,i]*w[j]/A[j])**(-5.0)
    
def Pi_check(i):    
    return sum((tau[l,i]*w[l]/A[l])**(-5.0) for l in iso3)

def trade_balance(i):
    return instance.w[i]*instance.L[i]*Pi_check(i)-sum([w[j]*L[j]*p_ji_check(j,i) for j in iso3])

### Save stored parameter values to dictionaries
w,L,tau,A = {},{},{},{}
iso3 = list(range(1,201))
for i in iso3:
    w[i] = instance.w[i]
    L[i] = instance.L[i]
    A[i] = instance.A[i].value
    for j in iso3:
        tau[i,j] = instance.tau[i,j]

Now let's loop over all trade balances for each country.

In [13]:
for i in iso3:
    print(trade_balance(i))

-0.0016460728510244715
-0.5303127686880127
0.15084406400446104
0.06428929409501505
0.06718743766050861
-0.6129938962813982
-0.39789020868038183
-0.2927348878956712
-0.3933787815665296
4.641834204731584
-0.44984761075217
-0.2568725690794877
0.9954279443252648
-0.15744580357394283
2.689587186718612
0.0755165938338711
-0.30370316752036325
-0.5461099954089141
-0.3446186278857986
0.28618823406504423
-0.42791187821556487
-0.08430386980993337
3.028835323981089
-0.36921925395302074
0.21462107571586364
-0.3571126633747228
-0.215387465576108
-0.4588521021218535
-0.5223105323564288
-0.1628579393039245
0.6893533767131833
-0.21489850932933136
11.463947239671642
-0.41203732836090795
1.2973915091048585
1.33138642266659
-0.09876626691029866
-0.5741400522813354
0.026082637695796995
-0.44110369775455205
1.2952822833740498
-0.34306859438255344
6.547479276123892
-0.3319455959003351
0.0726116224108585
-0.4484288530040106
-0.6314332496441288
-0.3867037344927697
1.4632720813253453
-0.0407114663964252
4.10953

The trade balances aren't quite zero, but in general are rather small. I'm okay with this especially given that we started with a huge objective function. You may want to try leaving other markets unconstrained to try to reach a smaller objective, but it's importance depends on how precise you need these parameters to be.

In this case, we are able to solve for close to optimal solution values. However, some models will be difficult to solve using the "IPOpt" solver (a free, non-linear solver), and sometimes commercial solvers are much, much better at finding solutions to particular types of large problems. Cplex, Gurobi, and Knitro are fast, commercial solvers that you can use instead, but they are not free. CONOPT, in particular, may be a good algorithm here, since we have many nonlinear constraints. I haven't figured out how to get these locally without paying, but we have an institutional subscription to GAMS, which includes (demos to some of) these and many more commercial solvers. Pyomo allows you to write your problem as a script readable by GAMS, which allows for use of its more powerful solvers (without having to code in GAMS). Be aware that many solvers require you to start with a small value of your objective function, so you may need to either scale down your variables and/or objective function by a constant, or solve it first locally and then export the problem later. Solvers may also require variable boundaries that others do not. To do so, run:

In [119]:
instance.write("goods_mkt_clearing.gms",io_options={'put_results':'dat'})

('goods_mkt_clearing.gms', 140433346716528)

Questions? Comments?

Email me: jsayre@berkeley.edu